# ６章. 学習に関するテクニック

## 6-1. パラメータの更新

最適化問題は難しい。確率的勾配降下法（SGD）は単純で、パラメータ空間を闇雲に探すよりも賢い方法だった。しかし、解決したい問題によってはSDGよりもさらにスマートな手法が存在する。

In [1]:
# -*- coding:utf-8 -*-

# SDG クラスの実装

class SDG:
    
    def __init__(self, lr=0.01):  # lr(learning rate) : 学習係数をインスタンスに持つ
        self.lr = lr
        
    def upgrade(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

SDGは関数の形状が等方的でないと、非効率な経路になる。

Momentum

モーメンタムとは運動量を意味し、物理と関係がある。

数式：v←av＊ｎ＊dL/dW, W←W＋v

dL/dW：Wに対する損失関数の勾配

W：更新する重みパラメータ

n：学習係数

v：物理でいうところの速度

av：物体が何も力を受けないときに徐々に減速するための役割を担う，物理でいう地面の摩擦や空気抵抗



In [2]:
# -*- coding:utf-8 -*-

# Momentum の実装

class Momentum:
    
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

Momentumを使って最適化問題を解くと、SDGよりも早く(0, 0)に向かって収束している様子が確認できる。これはx軸方向に受ける力は小さくても常に同じ方向の力を受けることで加速するためである。y軸方向は正と負の方向の力を交互に受けるため、それらは互いに打ち消し合い、y軸方向の速度が安定しない。これらによってSDGと比較してx軸方向へ早く進み、(0, 0)への収束が早まる。

AdaGrad

学習係数の減衰：学習が進むについて学習係数を小さくすること。最初に大きく学習し、次第に小さく学習するという手法。

AdaGradは一つ一つのパラメータに対して適応的に学習係数を調整しながら学習を行う手法。

数式：h←h＋dL/dW 二乗和 dL/DW,  W←W-n * 1/sqrt(h) * dL/dW

h：これまで経験した勾配の値を２乗和として保持する。パラメータの更新の際にかけ合わせている、1/sqrt(h)を乗算することは学習スケールの調整を担っており、これはパラメータの要素の中で大きく更新された要素は学習係数が次第に小さくなるという学習係数の減衰をパラメータの要素ごとに行うことができる。

In [7]:
# -*- coding;utf-8 -*-

# AdaGrad の実装

import numpy as np

class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)  # 1e-7　ゼロ除算を防ぐ

Adam

Momentumは、ボールがお椀を転がるように物理法則に準じる動きを表現し、AdaGradは、パラメータの要素ごとに適応的に更新ステップを調整した。この２つの手法を融合したすることがAdamのベースとなるアイデア。

Adamは2015年に提案された新しい手法であり、理論は複雑だが、先の２つの手法の利点を組み合わせることで効率的にパラメータ空間を探索することが期待される。

Adamは３つのハイパーパラメータを設定する。１．学習係数、２．一次モーメント用の係数b1、３．二次モーメント用の係数b2である。論文によるとb1は0.9, b2は0.999であり、その設定値は多くの場合うまく行くらしい。

MNISTデータセットによる更新手法の比較

５層ニューラルネットワーク、各層１００個のニューロン、活性化関数はReLUを使用したネットワーク。

（ソースコードは配賦済み）

実験は学習係数のハイパーパラメータや、ニューラルネットワークの構造（何層の深さかなど）によって結果は変化する。ただし、一般的にはSDGよりも他の３つの手法が早く学習でき、時には最終的な認識性能も高くなる。

## 6-2. 重みの初期値

## 6-3. Batch Normalization

## 6-4. 正則化

## 6-5. ハイパーパラメータの検証

## 6-6. まとめ